<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/JTC_v3_3_Hierarchial_model_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependency

In [1]:
! pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 881.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=af3cc1e52e071b6036878782616701b1b452e4578670171f1be432237328671a
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


# load packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
import json

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

from sentence_transformers import SentenceTransformer, util
from sklearn import metrics

In [4]:
from typing import Any, Optional

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
import pickle

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [6]:
from google.colab import drive
drive.mount('/content/drive')
root_path = "drive/MyDrive/trademe_data/"

Mounted at /content/drive


# load  data

In [7]:
raw_path = f"{root_path}raw.csv"
c3_dict_path = f"{root_path}c3_dict.json"
c1_dict_path = f"{root_path}c1_dict.json"

In [8]:
raw_df = pd.read_csv(raw_path)
raw_df.head(1)

,Unnamed: 0,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature
0,0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualifi asbesto remov


In [9]:
raw_df = raw_df[['y_cat_id', 'x_title_feature', 'cat_1', 'cat_2','cat_3']]
print(len(raw_df))
raw_df.head()

20000


,y_cat_id,x_title_feature,cat_1,cat_2,cat_3
0,5192,unqualifi asbesto remov,trades & services,labourers,labourers
1,5123,senior test analyst,it,testing,testing
2,6894,ict trainer supervisor,education,tutoring & training,tutoring & training
3,5197,automot specialist multi facet posit,trades & services,technicians,technicians
4,5114,busi analyst,it,business & systems analysts,business & systems analysts


### load level dicts

In [10]:
c3_dict = json.load(open(c3_dict_path))
print(len(c3_dict.keys()))
c3_ununique_node_c2_list = c3_dict.keys()
c3_ununique_node_c2_list

3


dict_keys(['accountants', 'drivers & couriers', 'nursing & midwifery'])

In [11]:
c1_dict = json.load(open(c1_dict_path))
print(len(c1_dict.keys()))
c1_ununique_node_c2_list = c1_dict.keys()
c1_ununique_node_c2_list

10


dict_keys(['administration', 'analysts', 'architects', 'drafting', 'electrical', 'health & safety', 'management', 'project management', 'security', 'supervisors & forepersons'])

# Model Classes

In [12]:
class BasicClassifierRFC:
    def __init__(self):
        self.clf = None

    def build(self):
        self.clf = RandomForestClassifier(n_estimators=100)
    
    def train(self, X_train, y_train):
        return self.clf.fit(X_train, y_train)

    def predict(self, input):
        return self.batch_predict([input])

    def batch_predict(self, inputs):
        return self.clf.predict(inputs)

## Root Model

In [35]:
class RootClassifier:
    def __init__(self, df, classifier=BasicClassifierRFC(), col_X="x_title_feature_encoding", col_y="cat_2", test_size=0.2, random_state=123):
        self.df = df
        self.classifier = classifier
        self.test_size = test_size
        self.random_state = random_state
        self.col_X = col_X
        self.col_y = col_y
        self.accuracy = {}
        self.reports = {}

    def _train_test_split(self):
        X_train, X_test , y_train, y_test = train_test_split(self.df[self.col_X],   self.df[[self.col_y]],  test_size=self.test_size, random_state=self.random_state)
        return X_train.to_list(), X_test.to_list() , y_train, y_test

    def build(self):
        X_train, X_test , y_train, y_test = self._train_test_split()
        self.classifier.build()
        model = self.classifier.train(X_train, y_train)
        # eval
        y_pred = self.classifier.batch_predict(X_test)
        self.accuracy['root'] = metrics.accuracy_score(y_test, y_pred)
        self.reports['root'] = classification_report(y_test, y_pred)
        return model

    def eval(self):
        return self.accuracy, self.reports

In [14]:
class LeavesClassifier:
    def __init__(self, df, root_nodes, col_y, col_X="x_title_feature_encoding", test_size=0.2, random_state=123):
        self.df = df
        self.root_nodes = root_nodes
        self.test_size = test_size
        self.random_state = random_state
        self.col_X = col_X
        self.col_y = col_y
        self.accuracy = {}
        self.reports = {}

    def _train_test_split(self, node):
        node_df = self.df[self.df.cat_2 == node]
        X_train, X_test , y_train, y_test = train_test_split(node_df[self.col_X],   node_df[[self.col_y]],  test_size=self.test_size, random_state=self.random_state)
        return X_train.to_list(), X_test.to_list() , y_train, y_test

    def build(self):
        leaves_models = {}
        for node in self.root_nodes:
            X_train, X_test , y_train, y_test = self._train_test_split(node)
            classifier = BasicClassifierRFC()
            classifier.build()
            leaves_models[node] = classifier.train(X_train, y_train)
            # eval
            y_pred = classifier.batch_predict(X_test)
            self.accuracy[node] = metrics.accuracy_score(y_test, y_pred)
            self.reports[node] = classification_report(y_test, y_pred)
        return leaves_models

    def eval(self):
        return self.accuracy, self.reports

## hierarchical model

In [70]:
class HierarchicalClassifier:
    def __init__(self, root_model, cat_3_models, cat_1_models):
        self.root_model = root_model
        self.cat_3_models = cat_3_models
        self.cat_1_models = cat_1_models
        
    def predict(self, query):
        query = [query]
        root_pred = self.root_model.predict(query)[0]
        print(f"root predict: {root_pred}")
        if root_pred in self.cat_3_models.keys():
            cat_3_predict = self.cat_3_models[root_pred].predict(query)
            print(cat_3_predict)
            return cat_3_predict
        elif root_pred in self.cat_1_models.keys():
            cat_1_predict = self.cat_1_models[root_pred].predict(query)
            print(cat_1_predict)
            return cat_1_predict

        return root_pred

    def batch_predict(self, queries):
        predictions = []
        for query in queries:
            predictions.append(self.predict([query]))
        return predictions

    

# Data Preparation

In [16]:
data_df = raw_df.copy()

## encoding

In [17]:
data_df['x_title_feature_encoding'] = data_df['x_title_feature'].apply(lambda x: model.encode(str(x), convert_to_tensor=True).cpu().numpy())

In [19]:
print(len(data_df))
data_df.head(2)

20000


,y_cat_id,x_title_feature,cat_1,cat_2,cat_3,x_title_feature_encoding
0,5192,unqualifi asbesto remov,trades & services,labourers,labourers,"[-0.04014635, 0.04006017, -0.052789543, 0.0232..."
1,5123,senior test analyst,it,testing,testing,"[-0.02413135, 0.055597737, -0.103181325, 0.024..."


## split for Hierarchial model and eval

In [21]:
train_dataset = data_df.sample(frac = 0.8)
print(len(train_dataset))

16000


In [22]:
eval_dataset = data_df.drop(train_dataset.index)
print(len(eval_dataset))

4000


# Model training for each level

## C1 models

In [23]:
c1_root_nodes = list(c1_dict.keys())
c1_root_nodes

['administration',
 'analysts',
 'architects',
 'drafting',
 'electrical',
 'health & safety',
 'management',
 'project management',
 'security',
 'supervisors & forepersons']

In [24]:
c1_classifiers = LeavesClassifier(df=data_df, root_nodes=c1_root_nodes, col_y="cat_1")
c1_models = c1_classifiers.build()

<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was 

In [25]:
acc, rep = c1_classifiers.eval()

In [26]:
for k in acc.keys():
    print(k)
    print(acc[k])
    print(rep[k])

administration
0.84
                         precision    recall  f1-score   support

             healthcare       0.75      0.26      0.39        34
office & administration       0.85      0.98      0.91       141

               accuracy                           0.84       175
              macro avg       0.80      0.62      0.65       175
           weighted avg       0.83      0.84      0.81       175

analysts
0.7142857142857143
                              precision    recall  f1-score   support

                  accounting       1.00      0.60      0.75         5
banking, finance & insurance       0.50      1.00      0.67         2

                    accuracy                           0.71         7
                   macro avg       0.75      0.80      0.71         7
                weighted avg       0.86      0.71      0.73         7

architects
0.9166666666666666
              precision    recall  f1-score   support

architecture       0.91      0.91      0.91        

## C3 models

In [27]:
c3_root_nodes = list(c3_dict.keys())
c3_root_nodes

['accountants', 'drivers & couriers', 'nursing & midwifery']

In [29]:
c3_classifiers = LeavesClassifier(df=data_df, root_nodes=c3_root_nodes, col_y="cat_3")
c3_models = c3_classifiers.build()

<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and be

In [30]:
acc, rep = c3_classifiers.eval()

In [31]:
for k in acc.keys():
    print(k)
    print(acc[k])

accountants
0.6875
drivers & couriers
0.9295774647887324
nursing & midwifery
0.6822429906542056


In [32]:
for k in rep.keys():
    print(k)
    print(rep[k])

accountants
                                  precision    recall  f1-score   support

           assistant accountants       0.67      0.86      0.75        14
                           audit       1.00      0.75      0.86         8
      business advisory services       0.62      0.50      0.56        10
financial accounting & reporting       0.62      0.77      0.69        13
                    general / ca       0.00      0.00      0.00         3
          management accountants       0.69      0.92      0.79        12
                             tax       0.00      0.00      0.00         4

                        accuracy                           0.69        64
                       macro avg       0.51      0.54      0.52        64
                    weighted avg       0.62      0.69      0.65        64

drivers & couriers
              precision    recall  f1-score   support

    couriers       1.00      0.60      0.75        25
    truck ht       0.92      1.00      0.96

## Root model

In [36]:
root_classifier = RootClassifier(df=data_df)
root_model = root_classifier.build()

<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and be

In [37]:
acc, rep = root_classifier.eval()

In [38]:
print(f"accuracy of root: {acc['root']}")

accuracy of root: 0.57975


In [39]:
print(rep['root'])

                                             precision    recall  f1-score   support

                         account management       0.25      0.11      0.15         9
                                accountants       0.75      0.80      0.77        70
                    accounts administrators       0.64      0.33      0.44        21
                           accounts payable       0.83      0.50      0.62        10
                        accounts receivable       1.00      0.40      0.57         5
                             administration       0.34      0.73      0.46       154
                                advertising       1.00      0.25      0.40         4
                   air con. & refrigeration       0.55      0.50      0.52        24
                                   analysts       1.00      0.43      0.60         7
                                 architects       0.71      0.81      0.76        21
                              area managers       0.00      0.00

# Hierarcial model

In [42]:
X_eval = eval_dataset["x_title_feature_encoding"].to_list()
y_eval = eval_dataset["y_cat_id"]

## eval of singal predict

In [72]:
hierarcial_classifier = HierarchicalClassifier(root_model= root_model, cat_3_models= c3_models, cat_1_models = c1_models)

In [73]:
preds = hierarcial_classifier.predict(X_eval[0])

root predict: bar staff & baristas


In [75]:
preds

'bar staff & baristas'

In [76]:
y_eval.head(1)

12    5098
Name: y_cat_id, dtype: int64

In [77]:
eval_dataset[eval_dataset.y_cat_id == 5098].cat_2.unique()

array(['bar staff & baristas'], dtype=object)